In [4]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from skimage import color
import random
import cv2  #librería reconocimiento de imágenes
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from PIL import Image
from sklearn.metrics import *
import time
from google.colab import drive
drive.mount('/content/drive')

ruta_maskwein = '/content/drive/MyDrive/clases ia/PROYECTO IA/Dataset/mask_weared_incorrect'
maskwein = []
img_size = 150

#hace lo mismo para cada una de las imagenes de la carpeta, escala de grises
for img in os.listdir(ruta_maskwein):
  img = cv2.imread(os.path.join(ruta_maskwein,img))
  img_gray_size = cv2.resize(img,(img_size,img_size))
  maskwein.append(img_gray_size)

maskwein = np.array(maskwein)

ruta_withmask = '/content/drive/MyDrive/clases ia/PROYECTO IA/Dataset/with_mask'
withmask = []
img_size = 150

#hace lo mismo para cada una de las imagenes de la carpeta
for img in os.listdir(ruta_withmask):
  img = cv2.imread(os.path.join(ruta_withmask,img))
  img_gray_size = cv2.resize(img,(img_size,img_size))
  withmask.append(img_gray_size)

withmask = np.array(withmask)

ruta_withoutmask = '/content/drive/MyDrive/clases ia/PROYECTO IA/Dataset/without_mask'
withoutmask = []
img_size = 150

#hace lo mismo para cada una de las imagenes de la carpeta
for img in os.listdir(ruta_withoutmask):
  img = cv2.imread(os.path.join(ruta_withoutmask,img))
  img_gray_size = cv2.resize(img,(img_size,img_size))
  withoutmask.append(img_gray_size)

withoutmask = np.array(withoutmask)

images = np.concatenate([maskwein, withmask, withoutmask])
images = np.array(images)

label_maskwein = np.repeat (0, 2994)
label_withmask = np.repeat (1, 2994)
label_withoutmask = np.repeat (2, 2994)

class_names = ['Mal uso', 'Uso correcto', 'No usa']

labels = np.concatenate ([label_maskwein, label_withmask, label_withoutmask])
labels = np.array(labels)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
X_train, X_test, y_train, y_test = train_test_split(images,labels, test_size=0.2, random_state=21)
x_train = X_train.reshape([-1, 150, 150, 1])
x_test = X_test.reshape([-1, 150, 150, 1])
x_train = X_train.astype('float32')
x_test = X_test.astype('float32')

x_train /= 255
x_test /= 255

In [6]:
y_train_ohe = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=3)

model_CNN =  tf.keras.models.Sequential()
model_CNN.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)))
model_CNN.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model_CNN.add(tf.keras.layers.MaxPooling2D((2, 2)))
model_CNN.add(tf.keras.layers.Flatten())
model_CNN.add(tf.keras.layers.Dense(64, activation='relu'))
model_CNN.add(tf.keras.layers.Dense(3, activation='softmax'))
model_CNN.summary()

model_CNN.compile(optimizer=tf.keras.optimizers.SGD(), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
inicio = time.time()

model_CNN.fit(x_train, y_train_ohe, epochs=5)
time.sleep(1)
fin = time.time()
print('Tiempo de ejecución ', fin-inicio)

test_loss, test_acc = model_CNN.evaluate(x_test, y_test_ohe)

print('Test accuracy:', test_acc)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 conv2d_3 (Conv2D)           (None, 150, 150, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 180000)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11520064  
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                      